# TRACK VISUALIZATION

In [ ]:
import fastf1
import random
import matplotlib as mpl
import numpy as np
from matplotlib import pyplot as plt
import plotly.graph_objects as go
from matplotlib.collections import LineCollection
from fastf1.core import Telemetry
import mplcursors

import logging

# Get the FastF1 logger and set its level to ERROR
fastf1_logger = logging.getLogger('fastf1')
fastf1_logger.setLevel(logging.ERROR)


In [ ]:
session = fastf1.get_session(2023, 'Bahrain','Race') # First race of the 2023 season
session.load()

In [ ]:
session.event

In [ ]:
session.laps

In [ ]:
for driver in session.drivers:
    broadcast_name = session.get_driver(driver)['BroadcastName']
    team_name = session.get_driver(driver)['TeamName']
    print(f"{driver:<10} : {broadcast_name:<20} : {team_name}")

### Telemetry car data for M Verstappen
* RPM: Revolutions Per Minute, the engine speed of the car's motor.
* Speed: Refers to the velocity of the car(km/h).
* nGear: Gear number the car is currently in.
* Throttle: The percentage of throttle pedal pressure applied by the driver, typically ranging from 0% (no throttle) to 100% (full throttle).
* Brake: Indicates whether the brakes are being applied (True) or not (False).
* DRS: Drag Reduction System, an aerodynamic device that reduces drag and increases speed on straights. It indicates whether the DRS is activated or not.
* Source: How the telemetry data was generated or obtained, such as directly from the car's sensors (car), positional data (pos), or through interpolation (interpolated).

In [ ]:
driver_no = '1'
car_1 = session.car_data[driver_no]
car_1

In [ ]:
car_1.columns

### Telemetry position data for M Verstappen
* Status: Status of the car's position, on track (OnTrack) or off track (OffTrack).
* X: The horizontal position of the car on the track, often measured in tenths of a meter.
* Y: The lateral position of the car on the track, typically measured in tenths of a meter.
* Z: The vertical position of the car relative to the track surface, usually measured in tenths of a meter.
* Source: Specifies how the positional data was obtained or generated, such as directly from the car's sensors or through interpolation.

In [ ]:
pos_1 = session.pos_data[driver_no]
pos_1

In [ ]:
laps_1 = session.laps.pick_driver(driver_no)
print(laps_1.columns)
laps_1

In [ ]:
fastest_lap_1 = session.laps.pick_driver(driver_no).pick_fastest()
print("Fastest Lap Number:", fastest_lap_1.LapNumber)
fastest_lap_1 

We can get telemetry data for the fastest lap

In [ ]:
fast_tel_1 = fastest_lap_1.get_telemetry()
fast_tel_1

Telemetry data of a driver for ALL laps (not just the fastest.)
In our case, this makes the most sense, as we will want to see the telemetry data from the timestamp which a radio conversation occurs! 

In [ ]:
all_laps_1 = session.laps.pick_driver(driver_no) #STARTING POINT OF LAP DATA
tel_all_laps_1 = all_laps_1.get_telemetry()
print(tel_all_laps_1.columns)
tel_all_laps_1

For now, let's act as if we have a list of timestamps from audio data, later on in the project we actually will have those timestamps, but now lets just check to see if we can mark the position of the car in the track according to the  timestamp.

In the audio data, the time stamp is given as
"Utc":"2023-04-02T05:11:40.709Z"
Meaning it is 2023-04-02 05:11:40.709

In the lap info we have from Fast F1- the equivalent version is **Date**
2023-03-05 14:01:02.640

So it would be wise to use this timestamp.

In [ ]:
#random.sample(list(tel_all_laps_1.Date),k=10)
rnd_timestamps = [timestamp.strftime('%Y-%m-%d %H:%M:%S.%f') for timestamp in random.sample(list(tel_all_laps_1.Date),k=30)]
rnd_timestamps

In [ ]:
tel_some_laps_1 = tel_all_laps_1[tel_all_laps_1['Date'].isin(rnd_timestamps)]
tel_some_laps_1

Let's plot the track

In [ ]:
colormap = mpl.cm.plasma

In [ ]:
x = tel_all_laps_1['X']              # values for x-axis
y = tel_all_laps_1['Y']              # values for y-axis
color = tel_all_laps_1['RPM']      # value to base color gradient on

In [ ]:
points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

In [ ]:
fig, ax = plt.subplots(sharex=True, sharey=True, figsize=(12, 6.75))

plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.12)
ax.axis('off')
ax.plot(tel_all_laps_1['X'], tel_all_laps_1['Y'], color='black', linestyle='-', linewidth=16, zorder=0)

# Map from data points to colors
norm = plt.Normalize(color.min(), color.max())
lc = LineCollection(segments, cmap=colormap, norm=norm, linestyle='-', linewidth=5)
lc.set_array(color)
line = ax.add_collection(lc)

# Mark points from tel_some_laps_1['Date']
for timestamp in tel_some_laps_1['Date']:
    # Find the index of the timestamp in tel_all_laps_1
    index = np.where(tel_all_laps_1['Date'] == timestamp)[0]
    if len(index) > 0:
        # Get the corresponding point on the track
        point_x = tel_all_laps_1['X'][index[0]]
        point_y = tel_all_laps_1['Y'][index[0]]
        ax.plot(point_x, point_y, marker='o', markersize=8, color='red')

In [ ]:
fig, ax = plt.subplots(sharex=True, sharey=True, figsize=(12, 6.75))

plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.12)
ax.axis('off')
ax.plot(tel_all_laps_1['X'], tel_all_laps_1['Y'], color='black', linestyle='-', linewidth=16, zorder=0)

norm = plt.Normalize(color.min(), color.max())
lc = LineCollection(segments, cmap=colormap, norm=norm, linestyle='-', linewidth=5)
lc.set_array(color)
line = ax.add_collection(lc)

for timestamp in tel_some_laps_1['Date']:
    index = np.where(tel_all_laps_1['Date'] == timestamp)[0]
    if len(index) > 0:
        # Get the corresponding point on the track
        point_x = tel_all_laps_1['X'][index[0]]
        point_y = tel_all_laps_1['Y'][index[0]]
        # Plot the point
        ax.plot(point_x, point_y, marker='o', markersize=8, color='red')
        # Annotate the timestamp
        ax.annotate(timestamp.strftime('%H:%M:%S'), (point_x, point_y),
                    textcoords="offset points", xytext=(10,10), ha='center')

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=tel_all_laps_1['X'],
    y=tel_all_laps_1['Y'],
    mode='lines',
    line=dict(color='black', width=16),
    name='Track',
    showlegend=False
))

for timestamp in tel_some_laps_1['Date']:
    index = np.where(tel_all_laps_1['Date'] == timestamp)[0]
    if len(index) > 0:
        i = index[0]
        point_x = tel_all_laps_1['X'][i] #36977 ______ [36977] index[0] vs index
        point_y = tel_all_laps_1['Y'][i]
        fig.add_trace(go.Scatter(
            x=[point_x],
            y=[point_y],
            mode='markers',
            marker=dict(size=8, color='red'),
            #text=[timestamp.strftime('%Y-%m-%d %H:%M:%S')],
            text = [f"Time:{timestamp.strftime('%Y-%m-%d %H:%M:%S')}"
                    f"<br>RPM:{tel_all_laps_1['RPM'][i]}"
                    f"<br>Brake:{tel_all_laps_1['Brake'][i]}"
                    f"<br>DRS:{tel_all_laps_1['DRS'][i]}"
                    f"<br>Throttle:{tel_all_laps_1['Throttle'][i]}"],
            hoverinfo='text',
            name=timestamp.strftime('%H:%M:%S')  
        ))

driver_name = session.laps.loc[session.laps['DriverNumber'] == f'{driver_no}', 'Driver'].iloc[0]

fig.update_layout(
    title=f"{session.event.EventName} Laps / Driver:{driver_no}-{driver_name}",
    xaxis=dict(title="X"),
    yaxis=dict(title="Y"),
    coloraxis_showscale=False
)

fig.show()


In [ ]:
tel_all_laps_1.columns

In [ ]:
# rpm, (yan yana)brake,drs, throttle

In [ ]:
timestamp = "2023-03-05 15:35:51.000"
tel_all_laps_1[tel_all_laps_1['Date'] >  timestamp]